In [72]:
import os
import pandas as pd

### 각자 해당 엑셀이 위치한 폴더 경로를 입력
# 해당 경로 내 파일 리스트 확인. 
folders = os.listdir('../Desktop/아파트(매매)_5개년실거래가_16.10.01_21.09.30')

# 해당 경로 이동
%cd ../Desktop/아파트(매매)_5개년실거래가_16.10.01_21.09.30

C:\Users\Admin\Desktop\아파트(매매)_5개년실거래가_16.10.01_21.09.30


In [78]:
# 여러 개의 csv를 불러와서 concat하는 loop문 생성

# DataFrame 생성
df_f = pd.DataFrame() 

for files in folders: 
    df = pd.read_excel(files) 
    df_f = pd.concat([df_f, df])

df_f.head()

,시군구,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도
0,서울특별시 강남구 개포동,개포2차현대아파트(220),77.75,201612,27,"80,000",5,1988
1,서울특별시 강남구 개포동,개포2차현대아파트(220),77.75,201703,15,"80,000",7,1988
2,서울특별시 강남구 개포동,개포2차현대아파트(220),77.75,201704,19,"85,000",2,1988
3,서울특별시 강남구 개포동,개포2차현대아파트(220),77.75,201707,8,"89,900",1,1988
4,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,79.97,201610,15,"99,000",4,1987


In [79]:
df_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406292 entries, 0 to 57086
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   시군구       406292 non-null  object 
 1   단지명       406292 non-null  object 
 2   전용면적(㎡)   406292 non-null  float64
 3   계약년월      406292 non-null  int64  
 4   계약일       406292 non-null  int64  
 5   거래금액(만원)  406292 non-null  object 
 6   층         406292 non-null  int64  
 7   건축년도      406292 non-null  int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 27.9+ MB


In [80]:
# 단지명, 계약일, 건축년도 등 필요없는 column 제거
df_f.drop(['단지명', '계약일', '층'], axis=1, inplace=True)

# 시군구에서 구만 분리
df_f['구'] = df_f['시군구'].str.split(' ', expand=True)[1]

# 시군구 column 제거
df_f.drop(['시군구'], axis=1, inplace=True)

# 구, 전용면적, 계약년월, 거래금액 순으로 columns 순서 재조정
df_f = df_f[['구', '전용면적(㎡)', '계약년월', '거래금액(만원)']]

df_f.head()

,구,전용면적(㎡),계약년월,거래금액(만원)
0,강남구,77.75,201612,"80,000"
1,강남구,77.75,201703,"80,000"
2,강남구,77.75,201704,"85,000"
3,강남구,77.75,201707,"89,900"
4,강남구,79.97,201610,"99,000"


In [81]:
# 거래금액의 ,를 제거
df_f['거래금액(만원)'] = df_f['거래금액(만원)'].apply(lambda x: x.replace(',', ''))

# 거래금액의 type을 ~로 변환
df_f['거래금액(만원)'] = pd.to_numeric(df_f['거래금액(만원)'])

# 거래금액을 평당가격으로 계산
df_f['평당가격'] = df_f['거래금액(만원)'] / df_f['전용면적(㎡)'] * 3.3

# 전용면적, 거래금액 column 제거
df_f.drop(['전용면적(㎡)', '거래금액(만원)'], axis=1, inplace=True)

df_f.head()

,구,계약년월,평당가격
0,강남구,201612,3395.498392
1,강남구,201703,3395.498392
2,강남구,201704,3607.717042
3,강남구,201707,3815.691318
4,강남구,201610,4085.281981


In [82]:
# 구와 계약년월을 기준으로 평균 가격을 구함
df1 = df_f.groupby(['구','계약년월'], as_index=False).mean()

# 계약년월을 string으로 변경?
df1['계약년월'] = df1['계약년월'].astype(str)

In [83]:
df1.head()

,구,계약년월,평당가격
0,강남구,201610,4265.262633
1,강남구,201611,3938.133281
2,강남구,201612,4294.448224
3,강남구,201701,4578.476984
4,강남구,201702,4761.424720


In [84]:
# 평당가격 => 월별평균평당가격 column 이름 변경
df1 = df1.rename(columns={'평당가격':'월별평균평당가격'})

# 가격의 소수점 0의 자리 반올림
df1 = df1.round(0)

df1.head()

,구,계약년월,월별평균평당가격
0,강남구,201610,4265.0
1,강남구,201611,3938.0
2,강남구,201612,4294.0
3,강남구,201701,4578.0
4,강남구,201702,4761.0


In [85]:
# 월별 평균 가격 1500개의 data 생성 확인
len(df1)

1500